<a href="https://colab.research.google.com/github/niteshctrl/mask_detection/blob/main/Face_Mask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import glob
import xml.etree.ElementTree as ET

from keras import Model
from keras.optimizers import Adam
from keras.layers import Flatten, Dense
from keras.applications import VGG19
from keras.applications import MobileNetV2
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_size = [224, 224]

anno_dir = 'annotations/'
images_dir = 'images/'
face_crops_dir = 'face_crops/'

In [ ]:
# The annotations directory has 853 XML files with nomenclature as "maksssksksss<_>.xml" where <_> stands for a
# number ranging from 0 to 852

# Similarly for images "maksssksksss<_>.png" for corresponding image file of XML

# Since we have 3 classes(with_mask, without_mask and mask_weared_incorrect), we will dump the face crops in
# three different directories after cropping the faces.

In [ ]:
j = 0
for i in range(853):
    tree = ET.parse(anno_dir+'maksssksksss'+str(i)+'.xml') # Parse the XML tree
    root = tree.getroot() # Get the root

    objects = root.findall('object')
    for obj in objects:                               # Iterate through all the objects(aka faces)
        name = obj.find('name').text
        
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        
        #########################
        # Crop and save the faces
        cropped = cv2.imread(images_dir+'maksssksksss'+str(i)+'.png')
        cropped = cropped[ymin:ymax, xmin:xmax]
        cv2.imwrite(face_crops_dir+name+'/'+str(j)+'.jpg', cropped)
        j = j + 1

In [ ]:
frame = cv2.imread('images/'+root.find('filename').text)
for i in range(len(name)):
    if name[i]=="without_mask":
        cv2.rectangle(frame,(int(xmin[i]), int(ymin[i])), (int(xmax[i]), int(ymax[i])), (0, 0, 255), 1)
    if name[i]=="with_mask":
        cv2.rectangle(frame,(int(xmin[i]), int(ymin[i])), (int(xmax[i]), int(ymax[i])), (0, 255, 0), 1)

cv2.imshow('Frame', frame)
cv2.waitKey(100000)
cv2.destroyAllWindows()

In [ ]:
classes = glob.glob(face_crops_dir+'*')
classes

['face_crops/mask_weared_incorrect',
 'face_crops/with_mask',
 'face_crops/without_mask']

In [ ]:
base_model = VGG19(weights='imagenet', 
                         include_top=False, 
                         input_shape=(224, 224, 3))

head_model = base_model.output
head_model = Flatten()(head_model)
head_model = Dense(128, activation='relu', kernel_initializer="he_normal")(head_model)
head_model = Dense(128, activation='relu', kernel_initializer="he_normal")(head_model)
head_model = Dense(3, activation='softmax')(head_model)

# Place the head model on top of the base model

model = Model(inputs=base_model.input, outputs=head_model)

In [ ]:
model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)     

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
optimizer = Adam(learning_rate=0.001)

model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

In [ ]:
generator = ImageDataGenerator(rotation_range=10, 
                               brightness_range=(0.5, 1.5), 
                               horizontal_flip=True,
                               preprocessing_function=preprocess_input,
                               validation_split=0.2
                              )

iterator = generator.flow_from_directory(face_crops_dir, 
                                         target_size=(224,224)
                                        )

Found 4072 images belonging to 3 classes.


In [ ]:
history = model.fit(iterator, epochs=5)

Epoch 1/5
 26/128 [=====>........................] - ETA: 6:22 - loss: 3.1318 - accuracy: 0.7200

KeyboardInterrupt: 